# Fetching the coordinates of neighbourhoods in Toronto

import libraries

In [1]:
import pandas as pd
import wikipedia as wp

We now load the data, but this time we've used wikipedia library to do so in a shorter way,
Then we process it further and get rid of NaN values of Neighborhood

In [6]:
html = wp.page("List of postal codes of Canada: M").html().encode("UTF-8")
df = pd.read_html(html, header = 0)[0]

df.head(10)

,Postal Code,Borough,Neighborhood
0,M1A,Not assigned,NaN
1,M2A,Not assigned,NaN
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"
5,M6A,North York,"Lawrence Manor, Lawrence Heights"
6,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"
7,M8A,Not assigned,NaN
8,M9A,Etobicoke,Islington Avenue
9,M1B,Scarborough,"Malvern, Rouge"


In [3]:
df_assigned = df[df.Borough != 'Not assigned']
df_assigned.reset_index(drop = True, inplace = True)

# Merge rows with the same Postcode
df_assigned = df_assigned.groupby(['Postal Code', 'Borough'], as_index=False).agg(lambda x: ', '.join(set(x)))

# Set 'Not assigned' Neigbourhood to be equal to Borough
na_nb_idx =df_assigned['Neighborhood'] == 'Not assigned'
df_assigned.loc[na_nb_idx, 'Neighborhood'] = df_assigned[na_nb_idx].Borough
df_assigned[df_assigned['Postal Code'] == 'M7A']

df_assigned.head(10)

,Postal Code,Borough,Neighborhood
0,M1B,Scarborough,"Malvern, Rouge"
1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae
5,M1J,Scarborough,Scarborough Village
6,M1K,Scarborough,"Kennedy Park, Ionview, East Birchmount Park"
7,M1L,Scarborough,"Golden Mile, Clairlea, Oakridge"
8,M1M,Scarborough,"Cliffside, Cliffcrest, Scarborough Village West"
9,M1N,Scarborough,"Birch Cliff, Cliffside West"


Now we load Geospatial Data for various postal codes in Toronto using the csv file

In [4]:
df_lon_lat = pd.read_csv('Geospatial_Coordinates.csv')
df_lon_lat.head(10)

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476
5,M1J,43.744734,-79.239476
6,M1K,43.727929,-79.262029
7,M1L,43.711112,-79.284577
8,M1M,43.716316,-79.239476
9,M1N,43.692657,-79.264848


Merging the two dataframes on postal codes gives us the longitude and latitude value of each neighborhood

In [5]:
Toronto_df = pd.merge(df_assigned,
                 df_lon_lat[['Postal Code','Latitude', 'Longitude']],
                 on='Postal Code')
Toronto_df.head(10)

,Postal Code,Borough,Neighborhood,Latitude,Longitude
0,M1B,Scarborough,"Malvern, Rouge",43.806686,-79.194353
1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476
5,M1J,Scarborough,Scarborough Village,43.744734,-79.239476
6,M1K,Scarborough,"Kennedy Park, Ionview, East Birchmount Park",43.727929,-79.262029
7,M1L,Scarborough,"Golden Mile, Clairlea, Oakridge",43.711112,-79.284577
8,M1M,Scarborough,"Cliffside, Cliffcrest, Scarborough Village West",43.716316,-79.239476
9,M1N,Scarborough,"Birch Cliff, Cliffside West",43.692657,-79.264848
